<a href="https://colab.research.google.com/github/CrazyMetoFQ/rhscra/blob/main/v3_RHscra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title init { display-mode: "form" }
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Image, clear_output, HTML

import os 

!pip install img2pdf
import img2pdf

from google.colab.output import eval_js

clear_output()

In [2]:
#@title gr { display-mode: "form" }

def get_reddit(subreddit,limit,timeframe=None,listing=None, user_agent_reddit = 'Chrome'):
  
  """
  returns request of json to reddit of following parameters
    subreddit --> str name of subreddit
    limit --> int max posts
    timeframe --> str, eg: hour, day, week, month, year, all
    listing --> str, eg: controversial, best, hot, new, random, rising, top
  """
  g_dict = {'subreddit':subreddit,'listing':listing, 'limit':limit, 'timeframe':timeframe} # given values
  t_dict = {'subreddit':'/r/{}','listing':'/{}','limit':'limit={}','timeframe':'&t={}'} # values to be formatted in 
  
  u_dict = {} # empty dict to be added to values that will be used
  for ((gk,gv),(tk,tv)) in zip(g_dict.items(),t_dict.items()):
    # formats the two dicts to be used if none then no format happens

    if gv == None:
      u_dict[gk] = ''
    else:
      u_dict[gk] = tv.format(gv)

  try:
      base_url = f"https://www.reddit.com{u_dict['subreddit']}{u_dict['listing']}.json?{u_dict['limit']}{u_dict['timeframe']}"
      r = requests.get(base_url, headers = {'User-agent': user_agent_reddit})
      print(base_url)
      if r.json().get('error'):
        raise Exception(r.json().get('message'))
  except Exception as e:
      print(e)
  return r



# get_images 

def get_files(rjsd, loc = '',
              allowed_types = [".jpg",".jpeg",".png"]):

  """
  r --> request json dict
  """

  if loc != '':
    if not os.path.isdir(loc):
      os.mkdir(loc)
    else:
      pass

  else:
    pass

  for (sno,post) in enumerate(rjsd['data']['children']):
    
    print(sno)
    p_url = post['data']['url'] # post
    p_title = post['data']['title'] # title
    p_text = post['data']['selftext']

    if any(fm in p_url for fm in allowed_types):
        print(p_url) # url of post
        print(p_title)
        print(p_text)


        rimg = requests.get(p_url)
        
        with open(f'{loc}/img{sno}.jpg', 'wb') as f: # writes img + serial number .jpg
          f.write(rimg.content)
        
        print('-'*10)

    else:
      print("not image")
      print(p_url) # url of post
      print(p_title)
      print(p_text)
      print('-'*10)



# converter
def convertimg(foldpath,pdfname, allowed=('.jpg','.png','.jpeg')):
    
    """
    convert all images in the folder to a pdf
    """
    
    imli = [f"{foldpath}/{fn}" for fn in os.listdir(foldpath)]
    
    with open(pdfname,"wb") as f:
        f.write(img2pdf.convert(imli))



In [3]:
#@title image class {display-mode: "form" }
class ImageViewEr:

  def __init__(self,image_path):

    self.img_path = image_path
    self.img_list = [f"{self.img_path}/{img}" for img in os.listdir(self.img_path)]

  def sort_ls(self, x, y):
    """
    truly sorts image list

    x ->  stars wih
    y -> ends with
    """

    imd = {}
    for img in self.img_list:

      no = int(img[ img.find(x) + len(x) : img.find(y) ])
      imd[no] = img
          
    sorted_chs = [imd[k] for k in sorted(imd)]

    self.sorted_images = sorted_chs
    return sorted_chs 


  def format_imgs(self):
    """
    converts the image of path to img objs,
    and adds breaks between each image
    """

    imdi = []
    for im in self.sorted_images:

      imdi.append(Image(im))
      imdi.append(HTML('<div> <br>---------------------------<br> <br> </div>'))

    self.formated_images = imdi

  
  def show_images(self):

    display(HTML("""
    <style>
    #output-body {
        display: flex;
        align-items: center;
        justify-content: center;
        flex-direction: column;
    }
    </style>
    """))
    display(*self.formated_images)




In [4]:
#@title AdvReddit
class advReddit:

  def __init__(self, subreddit, l):

    self.rjsd = get_reddit(subreddit, l).json()

  def imager(self):

    li = []
    for (sno,post) in enumerate(self.rjsd['data']['children']):
        
        try:
          print(sno)
          p_url = post['data']['url'] # post
          p_title = post['data']['title'] # title
          p_text = post['data']['selftext'] # text

          print(p_url)
          print(p_title)
          if any([fm in p_url for fm in [".jpg",".png"]]):
            h_img = Image(p_url)    
          else:
            h_img = HTML("<p>No Image</p>")

          h_title = HTML(f"<h3>{p_title}</h3>")
          h_text = HTML(f"<p>{p_text}</p>")
          h_buffer = HTML(f"<br><br><hr>")


          li.extend([HTML(str(sno)),h_title,h_img,h_text,h_buffer])
        except:
          pass
    
    self.imli = li


  def shw(self):

    display(HTML("""
      <style>
          #output-body {
              display: flex;
              align-items: center;
              justify-content: center;
              flex-direction: column;
          }
      </style>
    """))

    print("start")
    display(*self.imli)



In [ ]:
#@title Runner { display-mode: "form" }

subreddit = "teenagersnew" #@param {type:"string"}
limit = 10 #@param

tri = advReddit(subreddit, limit)

tri.imager()
tri.shw()

In [ ]:
#@title Choose settings [Old version]
subreddit = "memes" #@param {type:"string"}
limit =  40#@param {type:"integer"}
timeframe = None #@param ["None", "hour", "day", "week", "month", "year", "all"]
listing = None #@param ["None", "controversial", "best", "hot", "new", "random", "rising", "top"]

#@markdown ---
see_image = False #@param {type:"boolean"}
get_pdf = True #@param {type:"boolean"}
#@markdown <br>

base_path = "/content/meme" #@param
pdf_path = "tri.pdf" #@param {type:"string"}

#@markdown <br>
delete_prev = True #@param {type:"boolean"}

if not os.path.isdir(base_path):
  os.mkdir(base_path)
else:
  if delete_prev:
    for fn in os.listdir(base_path):
      os.remove(f"{base_path}/{fn}")
  else:
    pass


if timeframe == "None":
  timeframe = None
else:pass

if listing == "None":
  listing = None
else:pass


rjsd = get_reddit(subreddit,limit, timeframe,listing).json()
print(len([post for post in rjsd['data']['children']]))

get_files(rjsd,base_path)


if get_pdf:
  convertimg(base_path,pdf_path)
else:
  pass

if see_image:

  da_im = ImageViewEr(base_path)

  da_im.sort_ls("img",".jpg")
  da_im.format_imgs()

  da_im.show_images()
